# **Comparison of SOC Estimation Techniques for Lithium-Ion Batteries**
#### A sandbox aimed at gaining hands-on experience in assessing different methodologies and drawbacks.

In [1]:
# %load_ext autoreload
# %autoreload 2

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

import plotly.graph_objects as go
from plotly.subplots import make_subplots

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

The purpose of this notebook is to investigate and see implementation in e of various KFs and comapre their performances and associated metrics.

## Simulate a single phase
Firstly, we need data.<br>
In order to have full control, we will use PyBaMM to model synthetic data, as well as pseudo-OCV<>SOC lookup tables for our KFs.

In [2]:
from utils.notebook_utils import create_search_widget, create_dynamic_line_plots, create_single_fig_line_plot
from utils.data_generator import PyBAMMDataset

In [3]:
#====================================
PyBAMM_DS = PyBAMMDataset(
    capacity_Ah = 5, # Ah,
    initial_soc_percent = 10,
    cc_demand_A = -2,
    evaluation_time_step = 10, # s
    num_points = 1000 # datapoints
)
#====================================

sim = PyBAMM_DS.generate_sim_single_phase()

### Options to choose from for visualization of variables from sim
These are the options we can choose from from PyBaMM to visualize variables and adjsut settings.

Parameter settings that can be changed vefore solving a sim

In [4]:
create_search_widget(sim.parameter_values)

Text(value='', description='Search:', layout=Layout(width='300px'), placeholder='Type to search...')

Output()

Variables that can be called from sim solution to visualize

In [5]:
create_search_widget(sim.model.variable_names())

Text(value='', description='Search:', layout=Layout(width='300px'), placeholder='Type to search...')

Output()

<h4> Visualize simulation </h4>
Now, let's load data into variables for visualization.

In [21]:
entries_of_interest  = [
    "Time [s]",
    "Terminal voltage [V]",
    "Current [A]",
    "Cell temperature [K]",
    "Negative electrode stoichiometry",
    "Battery open-circuit voltage [V]",
    "Local ECM resistance [Ohm]"
]

tup = PyBAMM_DS.extract_entries_from_sim(entries_of_interest)

In [22]:
create_dynamic_line_plots(tup[1:], tup[0])

## SOC Estimation Methodology Comparison
There are several ways to go about SOC Estimation. There is the coulomb counting method, as well as an array of KF implementations with varying levels of complexity and consideratinos.<br>
Let's first look at the simplest one:
### Coulomb Counting

#### Basic Principle

Variations in charge (Q) during discharge/charge can be calculated as:

$$
Q(t) = Q(t-1) + \int_{t-1}^{t} I(t) \, dt
$$


Where:
- \( Q(t) \) is the charge at time \( t \).
- \( I(t) \) is the current at time \( t \).

The SoC can be expressed as:

$$
\text{SoC}(t) = \text{SoC}(t-1) + \frac{I(t)}{Q_{\text{rated}}} \Delta t
$$


#### Sources of Error

1. **Initial SoC Accuracy**: Errors in the initial SoC propagate through calculations.
2. **Current Measurement Error**: Inaccuracies in current measurement affect results.
3. **Integration Error**: Method of integrating current can introduce inaccuracies.
4. **Battery Capacity Uncertainty**: Fixed capacity assumptions may not hold over time.
5. **Timing Errors**: Inaccurate timing affects integration accuracy.

#### References

- Movassagh, K., et al. "A Critical Look at Coulomb Counting Approach for State of Charge Estimation in Batteries." Energies 2021.
- Analog Devices, "A Closer Look at State of Charge (SOC) and State of Health (SOH) Estimation Techniques for Batteries."


In [8]:
from models.soc_estimation import StateEstimator
SOCEstimator = StateEstimator(PyBAMM_DS)

OCV-SOC mapping located/retrieved from 'c:\Users\Javaid Baksh\Documents\Coding Projects\battery-state-estimation\datasets\ocv_soc_mapping_pybamm_5Ah.pkl'


In [9]:
soc_by_coulcnt = SOCEstimator.coulomb_counting()
create_single_fig_line_plot([soc_by_coulcnt], PyBAMM_DS.data.time, "SOC Estimation")

### Kalman Filters

Kalman Filters (KF) are algorithms used to estimate the state of a dynamic system from noisy measurements. It operates in two main phases: **Prediction** and **Update**.

##### Prediction Step

The prediction step estimates the current state based on the previous state:

$$

\hat{x}_{k|k-1} = F_k \hat{x}_{k-1|k-1} + B_k u_k
$$

Where:
- \( \hat{x}_{k|k-1} \): Predicted state estimate.
- \( F_k \): State transition matrix.
- \( B_k \): Control input matrix.
- \( u_k \): Control vector.

The predicted error covariance is updated as:

$$

P_{k|k-1} = F_k P_{k-1|k-1} F_k^T + Q_k
$$

##### Update Step

In the update step, the filter incorporates new measurements:

$$

K_k = P_{k|k-1} H_k^T (H_k P_{k|k-1} H_k^T + R_k)^{-1}
$$

Where:
- \( K_k \): Kalman gain.
- \( H_k \): Measurement matrix.
- \( R_k \): Measurement noise covariance.

The updated state estimate is given by:

$$

\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k (z_k - H_k \hat{x}_{k|k-1})
$$

#### Applications

Kalman Filters are widely used in:
- **Object Tracking**: Estimating position and velocity from noisy measurements.
- **Navigation**: Integrating data from various sensors to determine location and movement.
- **Control Systems**: Enhancing system performance by providing accurate state estimates.

#### References

For more details, refer to:
- Welch, G., & Bishop, G. (2006). "An Introduction to the Kalman Filter."


In [10]:
entries_of_interest  = [
    "Battery open-circuit voltage [V]",
    "Negative electrode stoichiometry",
]
tup = PyBAMM_DS.extract_entries_from_sim(entries_of_interest)

PyBAMM_DS.update_sim_vector(
    ocv = tup[0], 
    neg_elec_stoic = tup[1]
)

#### Visualizing comparisons across KFs
To compare different KF implementations, let's compare the following:
- SOC by coulomb counting
- SOC by a simple linear KF
- SOC by an extended linear KF
- "True" SOC by using ocv directly from PyBaMM model as input to a KF

In [11]:
soc_true = SOCEstimator.KF_true(
    P_initial=np.array([[1e0]]), # initial error covariance; affects starting point for P
    Q=np.array([[1e-4]]), # process noise covariance; affects P_pred > K > P_new ( how much the actual state of system can deviate from predicted state due to unmodeled influences or inherent randomness in system )
    R=np.array([[1e-2]]) # measurement noise covariance; affects K ( how much noise is expected in observed data )
)
soc_by_linKF = SOCEstimator.linKF_CCVSOC(
    P_initial=np.array([[1e0]]), # initial error covariance; affects starting point for P
    Q=np.array([[1e-1]]), # process noise covariance; affects P_pred > K > P_new ( how much the actual state of system can deviate from predicted state due to unmodeled influences or inherent randomness in system )
    R=np.array([[1e-2]]) # measurement noise covariance; affects K ( how much noise is expected in observed data )
)
soc_by_extCCV = SOCEstimator.extKF_CCVSOC(
    P_initial=np.array([[1e0]]), # initial error covariance; affects starting point for P
    Q=np.array([[1e-1]]), # process noise covariance; affects P_pred > K > P_new ( how much the actual state of system can deviate from predicted state due to unmodeled influences or inherent randomness in system )
    R=np.array([[1e-2]]) # measurement noise covariance; affects K ( how much noise is expected in observed data )
)

create_single_fig_line_plot([soc_by_coulcnt, soc_true, soc_by_linKF, soc_by_extCCV], PyBAMM_DS.data.time, "SOC Estimation")

Let's further breakdown the determination of these estimates by tracking and visualizing core components throughout the processs:

In [23]:
others_to_plot = []
for k,v in soc_by_linKF["other"].items():
    try:
        v = [arr.item() for arr in v]
    except: pass
    others_to_plot.append({"label":k, "data":v})
create_single_fig_line_plot(others_to_plot[1:], others_to_plot[0], "SOC Estimation")


#### Assessing impact of process noise covariance on SOC error
Dependence of KFs on initializations is a drawback in implementation as it introduces bias to these parameters rather than generalized to input signals. <br>
Let's look at how just changing covariances for process and signal noise alone can change the output of these estimates via assessing its SOC error (true-estimated).

In [25]:
soc_errs = []

soc_true = SOCEstimator.KF_true(
    P_initial=np.array([[1e0]]), # initial error covariance; affects starting point for P
    Q=np.array([[1e-4]]), # process noise covariance; affects P_pred > K > P_new ( how much the actual state of system can deviate from predicted state due to unmodeled influences or inherent randomness in system )
    R=np.array([[1e-2]]) # measurement noise covariance; affects K ( how much noise is expected in observed data )
)

for Q_factor in range(-5,0):
    for R_factor in [-1, -2]:
        soc_by_extCCV = SOCEstimator.extKF_CCVSOC(
            P_initial=np.array([[1e0]]), # initial error covariance; affects starting point for P
            Q=np.array([[1*10**Q_factor]]), # process noise covariance; affects P_pred > K > P_new ( how much the actual state of system can deviate from predicted state due to unmodeled influences or inherent randomness in system )
            R=np.array([[1*10**R_factor]]) # measurement noise covariance; affects K ( how much noise is expected in observed data )
        )
        soc_err_data = [a - b for a, b in zip(soc_true["data"], soc_by_extCCV["data"])]
        soc_errs.append({
            'label': f"Q ({Q_factor}) | R ({R_factor})",
            'data': soc_err_data
        })


create_single_fig_line_plot(soc_errs, PyBAMM_DS.data.time, "SOC Error Comparison")